In [1]:
import warnings 
warnings.filterwarnings('ignore')

## 머신러닝1 
- 한국에 위치한 A자동차 회사에서 해외로 부터 부품을 수입해온다.
- 20년 1월부터 25년 3월까지의 수입이벤트가 발생한 순간의 데이터를 나타낸다.
- m1.csv
    - country : 수입국가 (4개국, europe,india,japan,usa)
    - part_cateogry: 부품 카테고리
    - unit_price : 부품 가격 (수입국가의 화폐 단위에 맞춰짐)
    - quantity : 부품수량
    - 참고
        - 같은 부품이라도 시간에 따라 가격이 달라질 수 있다.

- m1_exchange.csv
    - 20년 1월부터 25년 3월까지의 환율정보가 담겨있다. (각국가의 기준단위에 대한 krw(원) 가격)
    - type : 국가 정보 (jpy 1엔, usd 1달러, eur 1유로, inr 1루피)   

In [2]:
import pandas as pd

df =pd.read_csv('C:/Users/i2max-DoeunKim/adps/EP_3/m1.csv')
exc =pd.read_csv('C:/Users/i2max-DoeunKim/adps/EP_3/m1_exchange.csv')

In [3]:
df.sample(5)

,timestamp,country,part_category,unit_price,quantity,part_ID
16202,2023-06-30 03:52:00,USA,Engine,876.63,6,IDS_7552
21781,2024-11-29 04:39:00,Japan,Brake,79133.08,1,IDS_2409
11531,2022-06-11 18:36:00,Japan,Electronics,101449.96,6,IDS_3605
8325,2021-10-22 03:43:00,India,Electronics,45303.66,5,IDS_5456
19603,2024-06-01 20:30:00,India,Interior,7240.37,156,IDS_7489


In [4]:
exc.sample(5)

,일자,기준환율,type
1850,2022.10.12,9.8296,jpy-krw
4547,2021.11.08,15.9700,inr-krw
912,2021.07.28,1150.2000,usd-krw
651,2022.08.18,1309.8000,usd-krw
1264,2025.02.28,9.6182,jpy-krw


In [5]:
# 환율 데이터 처리 
exc['일자'] = pd.to_datetime(exc['일자'])

# 각 나라별 환율 
exc.groupby('type').agg(mi = ('일자', min) , max = ('일자' , max))

,mi,max
type,,
eur-krw,2020-04-13,2025-04-14
inr-krw,2020-04-13,2025-04-14
jpy-krw,2020-04-13,2025-04-14
usd-krw,2020-04-13,2025-04-14


In [6]:

res = []
for t in exc['type'].unique() : 
    target = exc[exc['type'] == t] 

    new_date = pd.date_range(start = target['일자'].min() , end = target['일자'].max()) 
    date_df =pd.DataFrame({'new_date' : new_date})
    date_df['type'] = t

    merge_df = pd.merge(target, date_df , how = 'right', left_on = ['일자','type'], right_on = ['new_date','type'])
    merge_df = merge_df.fillna(method = 'ffill')
    merge_df['new_date'] = pd.to_datetime(merge_df['new_date'])

    country_mapper = {
        'jpy-krw' : 'Japan',
        'inr-krw' : 'India',
        'eur-krw' : 'Europe',
        'usd-krw' : 'USA'
    }

    merge_df['con'] = merge_df['type'].map(country_mapper)
    res.append(merge_df) 

m_df = pd.concat(res)
m_df.head(4)
m_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7312 entries, 0 to 1827
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   일자        7312 non-null   datetime64[ns]
 1   기준환율      7312 non-null   float64       
 2   type      7312 non-null   object        
 3   new_date  7312 non-null   datetime64[ns]
 4   con       7312 non-null   object        
dtypes: datetime64[ns](2), float64(1), object(2)
memory usage: 342.8+ KB


In [7]:
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['date'] = pd.to_datetime(df['timestamp'].dt.date)


# 환율 날짜의 최소값보다 큰 데이터만 불러오기 
t = df.loc[df['date'] >= m_df['new_date'].min()].reset_index(drop = True) 

# 환율데이터의 일자에 값이 있으면 0 (평일) 없으면 1 (주말)
t['주말'] = t['date'].map(lambda x : 0 if x in m_df['일자'].values else 1)

# 13시 이전 이후, 
t['hour13'] = t['timestamp'].dt.hour.map(lambda x : 1 if x >= 13 else 0)

In [8]:
f_df = pd.merge(t, m_df, how = 'left' , left_on = ['date','country'], right_on = ['new_date','con'])
f_df.head(3)

,timestamp,country,part_category,unit_price,quantity,part_ID,date,주말,hour13,일자,기준환율,type,new_date,con
0,2020-04-13 07:04:00,Japan,Brake,35338.59,45,IDS_6516,2020-04-13,0,0,2020-04-13,11.1862,jpy-krw,2020-04-13,Japan
1,2020-04-13 02:52:00,Japan,Electronics,97732.81,6,IDS_3605,2020-04-13,0,0,2020-04-13,11.1862,jpy-krw,2020-04-13,Japan
2,2020-04-13 05:16:00,India,Engine,10890.93,14,IDS_2209,2020-04-13,0,0,2020-04-13,15.9800,inr-krw,2020-04-13,India


In [9]:
import numpy as np 

def change_ex(x) : 
    weekend = x['주말']
    date = x['new_date']     #새로 만든 환율 날짜 기준
    hour = x['hour13']
    country = x['con']       #새로 만든 환율의 나라

    # 환율정보저장
    values = np.nan 

    # 평일(0)일때 같은 국가 + 같은 날짜의 환율을 찾아라 
    if weekend == 0 : 
        target = f_df[(f_df['con'] == country) & (f_df['new_date'] == date)]
        # 해당 날짜에 환율 정보가 있으면 고대로 사용
        if not target.empty : 
            values = target['기준환율'].values[0]

        else : 
            target = f_df[f_df['con'] == country].sort_values('new_date')
            # 해당 날짜에 정보가 없으면, new_date로 정렬해서 이전 값 중 마지막(iloc[-1])
            values = target[target['new_date'] <= date]['기준환율'].ffill().iloc[-1]

    # 주말 (1)일떄 
    else : 
        # 13이후일 때 다음 날 
        if hour == 1 : 
            # 다음날 환율 기준
            target_date = date + pd.Timedelta(days = 1) 
            # 그 전달 환율 기준
        else : 
            target_date = date - pd.Timedelta(days = 1)

        target = f_df[(f_df['new_date'] == target_date) & (f_df['con'] == country)] 
        # 있으면 그냥 쓰고 
        if not target.empty : 
            values = target['기준환율'].values[0]
        else : 
            # 해당 국가에서 날짜순으로 정렬된 데이터에서 
            target = f_df[f_df['con'] == country].sort_values('new_date')
            # 13시 이후 이면 [다음날]
            if hour == 1 : 
                # 가장 가까운 미래값 bfill의 iloc[0] 첫 번쨰 값 
                values = target[target['new_date'] >= date]['기준환율'].bfill().iloc[0]
            else : 
                # 같거나 이전인 데이터인 경우는 ffill() 이전 환율의 마지막 값 iloc[-1]
                values = target[target['new_date'] <= date]['기준환율'].ffill().iloc[-1]

    return values 

f_df['exchange'] = f_df[['주말','hour13','con','new_date']].apply(change_ex, axis = 1)

In [10]:
f_df.groupby('country')['exchange'].sum()

country
Europe    3.099508e+06
India     1.394184e+05
Japan     6.346799e+04
USA       5.627300e+06
Name: exchange, dtype: float64

In [11]:
f_df['주말'].value_counts()

주말
0    16091
1     5757
Name: count, dtype: int64

In [12]:
f_df['hour13'].value_counts()

hour13
0    11971
1     9877
Name: count, dtype: int64

머신러닝1     
아래의 기준으로 전처리를 시행하라    
1. 각 부품의 발주건에 대한 총수입금액 컬럼(total_price)을 만들고 계산하려한다..   
    a. 국가별 관세 비율은 다음과 같다. 미국(25%), 인도(5%), 유럽(10%), 일본(3%)     
    b. 일자별 환율은 공휴일 및 주말은 발표되지 않는다.     
    c. 공휴일 및 주말에 구매한 건에 대한 환율 계산은 13:00 이전에는 전날의 환율을, 13:00 이후에는 다음날의 환율을 적용한다.    
       (연속으로 쉬는 경우에도 그 다음 영업일의 환율을 반영)      
    d. A자동차가 부품에 대해 총수입금액은 unit_price * quantity * (1+ 관세/100) * 일자별 환율과 같다.   

In [16]:
# 환율 계산 
def cal_price(x) : 
    con = x['country'] 
    unit = x['unit_price']
    q = x['quantity']
    value = x['exchange']

    ratio = {'Japan' :3, 'India' :5, 'Europe' : 10, 'USA' : 25}
    return unit * q * (1 + ratio[con] / 100) * value 

f_df['price'] = f_df[['country','unit_price','quantity','exchange']].apply(cal_price, axis =1)
f_df['price'] = f_df['price'] 

머신러닝3    
일자별 3국(미국,유럽,인도)수입컬럼 금액 및 파생변수를 통해 일본에서 수입한 금액을 예측하는 모델링을 진행하려한다.       
아래 기준으로 데이터를 분할하고 모델링을 진행하라.    

1. 데이터 전처리
    (파생변수생성)
    - 일자별 3국의 수입국가 개수, 카테고리별 종류 수, 카테고리별 제품 수
    - 종속변수 : 일본의 수입금액 (독립변수가 존재하지 않는 일자는 제거)
    - 시간 관련 파생변수 4개
2. 모델링
    - 3가지 회귀모델
    - 평가지표 rmse
3. 데이터 분할
    - 일자 기준으로 정렬 후 행의 index기준으로(0부터시작) 3의 배수이거나 4의 배수이거나 5의 배수라면 train 그 외의 경우 test로 진행

In [18]:
w = f_df[f_df['주말'] == 1][['timestamp', 'price' ,'주말', 'hour13']]
w['price'] = (pd.to_numeric(w['price'])).astype(int)
w.head(3)

,timestamp,price,주말,hour13
12,2020-04-15 13:15:00,9172944,1,1
13,2020-04-15 01:06:00,12366163,1,0
14,2020-04-15 11:12:00,7921763,1,0


In [33]:
# 일본 뺴고 
df3 = f_df[f_df['country'] != 'Janpan'].reset_index(drop = True)

# 일자별 3국의 수입국가 개수
t1 = df3.groupby(['date','country']).size().unstack().fillna(0).astype('int')

# 카테고리별 종류 수
t2 = df3.groupby(['date','part_category']).size().unstack().fillna(0).astype(int)

# 카테고리별 제품 수
t3 = df3.groupby(['date','part_category'])['quantity'].sum().unstack().fillna(0).astype(int)

t = pd.concat([t1, t2, t3], axis = 1)
t.head()

,Europe,India,Japan,USA,Brake,Electronics,Engine,Interior,Suspension,Brake,Electronics,Engine,Interior,Suspension
date,,,,,,,,,,,,,,
2020-04-13,0,2,6,0,3,1,2,0,2,99,6,24,0,8
2020-04-14,1,1,2,0,2,1,0,1,0,39,33,0,138,0
2020-04-15,2,3,2,2,3,0,2,1,3,103,0,49,27,44
2020-04-16,0,1,2,1,2,0,1,1,0,198,0,21,10,0
2020-04-17,1,3,6,2,4,0,2,2,4,108,0,44,49,126


In [38]:
# 종속변수 : 일본의 수입금액 (독립변수가 존재하지 않는 일자는 제거) 
y = f_df[f_df['country'] =='Japan'].groupby(['date'])['price'].sum()
model_df =pd.concat([t, y], axis = 1).dropna().reset_index()



## 통계1
- 주택 가격 데이터이다. (종속변수 price)
- 3차 다항회귀 모델을 학습하고 평가하라.
- 독립변수간의 상호작용 항만을 고려한 경우와 그렇지 않은 경우의 r2값을 포함한 모델 결과 해석을 진행하라.

In [39]:
df =pd.read_csv('C:/Users/i2max-DoeunKim/adps/EP_3/s1.csv')
df.head()


,Size,Rooms,Year_Built,Distance_to_Park,Price
0,87.454012,5,2004.5,2.13,1158.64
1,145.071431,4,1982.7,4.33,1084.19
2,123.199394,3,1981.6,4.62,943.60
3,109.865848,4,1985.8,2.38,1185.59
4,65.601864,5,1980.6,2.46,1061.23


In [44]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold, cross_val_score

x = df.drop(columns =['Price'])
y = df['Price']


# ----------------------------
# 3) Interactions only (상호작용만)
# ---------------------------
poly = PolynomialFeatures(degree=3,include_bias = False, interaction_only = True) # 몇차항으로 만들건지에 대한 옵션
x_poly = poly.fit_transform(x)

# 변경한 3차항을 모델에 추가
model = LinearRegression()
model.fit(x_poly,y)
inter_r2 = model.score(x_poly, y)

cv = KFold(n_splits=5, shuffle=True, random_state=42)
inter_cv = cross_val_score(LinearRegression(), x_poly, y, cv=cv, scoring='r2')
inter_cv

array([0.64258544, 0.74576569, 0.82485545, 0.76823685, 0.57500215])

In [46]:
# ----------------------------
# 2) Full Polynomial (degree=3, 상호작용 포함)
# interaction = False : 상호작용항 + 자기자신 제곱/세제곱 항 모두 포함
# ----------------------------
poly_full = PolynomialFeatures(degree=3, include_bias=False, interaction_only=False)
X_full = poly_full.fit_transform(x)

lr_full = LinearRegression().fit(X_full, y)
full_r2 = lr_full.score(X_full, y)

cv = KFold(n_splits=5, shuffle=True, random_state=42)
full_cv = cross_val_score(LinearRegression(), X_full, y, cv=cv, scoring='r2')


In [47]:
inter_r2

0.76605543369704

In [48]:
full_r2

0.8152885507802708